# Groupby on Dataframes

In [3]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [4]:
#Let's make a dframe
dframe = DataFrame({'k1':['X','X','Y','Y','Z'],
                    'k2':['alpha','beta','alpha','beta','alpha'],
                    'dataset1':np.random.randn(5),
                    'dataset2':np.random.randn(5)})

#Show
dframe

,k1,k2,dataset1,dataset2
0,X,alpha,0.530376,-0.184457
1,X,beta,1.194088,0.668244
2,Y,alpha,-0.569268,0.915162
3,Y,beta,0.277660,1.050399
4,Z,alpha,-0.485894,0.728235


In [5]:
#Now let's see how to use groupby

#Lets grab the dataset1 column and group it by the k1 key
group1 = dframe['dataset1'].groupby(dframe['k1'])

#Show the groupby object
group1

In [6]:
#Now we can perform operations on this particular group
group1.mean()

k1
X    0.862232
Y   -0.145804
Z   -0.485894
Name: dataset1, dtype: float64

In [7]:
# We can use group keys that are series as well

#For example:

#We'll make some arrays for use as keys
cities = np.array(['NY','LA','LA','NY','NY'])
month = np.array(['JAN','FEB','JAN','FEB','JAN'])

#Now using the data from dataset1, group the means by city and month
dframe['dataset1'].groupby([cities,month]).mean()

LA  FEB    1.194088
    JAN   -0.569268
NY  FEB    0.277660
    JAN    0.022241
Name: dataset1, dtype: float64

In [8]:
# WE can also pass column names as group keys
dframe.groupby('k1').mean()

,dataset1,dataset2
k1,,
X,0.862232,0.241894
Y,-0.145804,0.982781
Z,-0.485894,0.728235


In [9]:
# Or multiple column names
dframe.groupby(['k2','k1']).mean() #position of k1 & k2 in snytax matters

dataset1  dataset2
k2    k1                    
alpha X   0.530376 -0.184457
      Y  -0.569268  0.915162
      Z  -0.485894  0.728235
beta  X   1.194088  0.668244
      Y   0.277660  1.050399

In [10]:
# Another useful groupby method is getting the group sizes
dframe.groupby(['k1']).size()

k1
X    2
Y    2
Z    1
dtype: int64

In [11]:
# We can also iterate over groups

#For example:
for name,group in dframe.groupby('k1'):
    print ("This is the %s group" %name)
    print (group)
    print ('\n')

This is the X group
  k1     k2  dataset1  dataset2
0  X  alpha  0.530376 -0.184457
1  X   beta  1.194088  0.668244


This is the Y group
  k1     k2  dataset1  dataset2
2  Y  alpha -0.569268  0.915162
3  Y   beta  0.277660  1.050399


This is the Z group
  k1     k2  dataset1  dataset2
4  Z  alpha -0.485894  0.728235




In [15]:
# A possibly useful tactic is creating a dictionary of the data pieces 
group_dict = dict(list(dframe.groupby('k1')))

#Show the group with Y
group_dict['Y']

,k1,k2,dataset1,dataset2
2,Y,alpha,-0.569268,0.915162
3,Y,beta,0.277660,1.050399


In [16]:
# We could have also chosen to do this with axis = 1

# Let's creat a dictionary for dtypes of objects!
group_dict_axis1 = dict(list(dframe.groupby(dframe.dtypes,axis=1)))

#show
group_dict_axis1

{dtype('float64'):    dataset1  dataset2
 0  0.530376 -0.184457
 1  1.194088  0.668244
 2 -0.569268  0.915162
 3  0.277660  1.050399
 4 -0.485894  0.728235, dtype('O'):   k1     k2
 0  X  alpha
 1  X   beta
 2  Y  alpha
 3  Y   beta
 4  Z  alpha}

In [17]:
# For example if we only wanted to group the dataset2 column with both sets of keys
dataset2_group = dframe.groupby(['k1','k2'])[['dataset2']]

dataset2_group.mean()

dataset2
k1 k2             
X  alpha -0.184457
   beta   0.668244
Y  alpha  0.915162
   beta   1.050399
Z  alpha  0.728235

# Grouping with dictionaries and series!

In [21]:
# Let's make a Dframe

animals = DataFrame(np.arange(16).reshape(4, 4),
                   columns=['W', 'X', 'Y', 'Z'],
                   index=['Dog', 'Cat', 'Bird', 'Mouse'])

#Now lets add some NAN values
animals.loc[1:2, ['W', 'Y']] = np.nan 

#Show
animals

,W,X,Y,Z
Dog,0.0,1,2.0,3
Cat,NaN,5,NaN,7
Bird,8.0,9,10.0,11
Mouse,12.0,13,14.0,15


In [23]:
# Now let's say I had a dictionary with behavior values in it
behavior_map = {'W': 'good', 'X': 'bad', 'Y': 'good','Z': 'bad'}


# Now we can groupby using that mapping
animal_col = animals.groupby(behavior_map, axis=1)

# Show the sum accroding to the groupby with the mapping
animal_col.sum()

# For example [dog][good] = [dog][Y]+[dog][W]

,bad,good
Dog,4.0,2.0
Cat,12.0,0.0
Bird,20.0,18.0
Mouse,28.0,26.0


In [25]:
# Now let's try it with a Series
behav_series = Series(behavior_map)

#Show
behav_series

W    good
X     bad
Y    good
Z     bad
dtype: object

In [26]:
# Now let's groupby the Series

animals.groupby(behav_series, axis=1).count()

,bad,good
Dog,2,2
Cat,2,0
Bird,2,2
Mouse,2,2


In [27]:
# Lets assume we wanted to group by the length of the animal names, we can pass the len function into groupby!

# Show
animals.groupby(len).sum() # cat + dog gets added

#Note the index is now number of letters in the animal name

,W,X,Y,Z
3,0.0,6,2.0,10
4,8.0,9,10.0,11
5,12.0,13,14.0,15


In [28]:
# We can also mix functions with arrays,dicts, and Series for groupby methods

# Set a list for keys
keys = ['A', 'B', 'A', 'B']

# Now groupby length of name and the keys to show max values
animals.groupby([len, keys]).max()

W   X     Y   Z
3 A   0.0   1   2.0   3
  B   NaN   5   NaN   7
4 A   8.0   9  10.0  11
5 B  12.0  13  14.0  15

In [29]:
# We can also use groupby with hierarchaly index levels

#Create a hierarchal column index
hier_col = pd.MultiIndex.from_arrays([['NY','NY','NY','SF','SF'],[1,2,3,1,2]],names=['City','sub_value'])

# Create a dframe with hierarchal index
dframe_hr = DataFrame(np.arange(25).reshape(5,5),columns=hier_col)

#Multiply values by 100 for clarity
dframe_hr = dframe_hr*100

#Show
dframe_hr

City         NY                SF      
sub_value     1     2     3     1     2
0             0   100   200   300   400
1           500   600   700   800   900
2          1000  1100  1200  1300  1400
3          1500  1600  1700  1800  1900
4          2000  2100  2200  2300  2400

# Data Aggregation!!

In [31]:
# Data Agrregation consists of operations that result in a scalar (e.g. mean(),sum(),count(), etc)

#Let's get a csv data set to play with
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/'


# Save the winquality.csv file in the same folder as your ipython notebooks, note the delimiter used ;
dframe_wine = pd.read_csv('winequality-red.csv',sep=';')

FileNotFoundError: File b'winequality-red.csv' does not exist

In [32]:
# Let's create a quick data set
from StringIO import StringIO

data ="""\
Sample   Animal   Intelligence
1        Dog     Smart
2 Dog Smart
3 Cat Dumb
4 Cat Dumb
5 Dog Dumb
6 Cat Smart"""

#Store as dframe
dframe = pd.read_table(StringIO(data),sep='\s+')

ModuleNotFoundError: No module named 'StringIO'